In [17]:
# %% [markdown]
# # HyperKZG Polynomial-Commitment Scheme – Interactive Walk-Through  
#  
# 
# **Dependencies**
# ```bash
# pip install sympy galois   # (galois optional – just pretty finite-field ints)
# ```
# We use a tiny prime field **𝔽₁₀₁** so numbers stay small and visual.  

# %%
import random, sympy as sp
random.seed(0)

# ───────────────────────────────────────  field wrapper  ────────────────────
try:
    import galois                           
    F = galois.GF(101)
    GF = lambda x: F(x)
except ImportError:                         
    P = 101
    class _F(int):
        def __new__(cls, x): return int.__new__(cls, x % P)
        def _cast(self, other): return other if isinstance(other, _F) else _F(other)
        def __add__(self, o): return _F(int(self)+int(o))
        __radd__ = __add__
        def __sub__(self, o): return _F(int(self)-int(o))
        __rsub__ = lambda s,o: _F(int(o)-int(s))
        def __mul__(self, o): return _F(int(self)*int(o))
        __rmul__ = __mul__
        def __pow__(self, e, *_): return _F(pow(int(self), e, P))
        def inv(self): return _F(pow(int(self), -1, P))
        def __truediv__(self, o): return self * self._cast(o).inv()
        __rtruediv__ = lambda s,o: _F(o) / s
    GF = lambda x: _F(x)

sp.init_printing()

# helper: evaluate SymPy poly at field element
def eval_poly(poly, x):
    return GF(int(sp.poly(poly).eval(int(x))))

# %% [markdown]
# ---
# ## Section 1  Gemini Even/Odd Folding (coefficient form)
# 
# We’ll build a random multilinear extension for **n = 3** (eight coefficients),
# compute its even/odd decomposition, fold once, and check the identity  
# \[
# f^{(1)}(X^{2}) \;=\;
# \tfrac12(f^{(0)}(X)+f^{(0)}(-X)) \;+\;
# u_0\,\tfrac12\bigl(f^{(0)}(X)-f^{(0)}(-X)\bigr)\!/X .
# \]

# %%
print("── Section 1 demo ──")
n, N = 3, 2**3
f_coeffs = [GF(random.randrange(1, 100)) for _ in range(N)]
X = sp.symbols('X')
f0 = sum(int(c)*X**i for i, c in enumerate(f_coeffs))
print("f⁰(X) =", f0)

u0 = GF(1)
even = sp.expand((f0.subs(X,  X) + f0.subs(X, -X)) / 2)
odd  = sp.expand((f0.subs(X,  X) - f0.subs(X, -X)) / (2*X))
print("even part =", even)
print("odd  part =", odd)
print("u₀ =", u0)

f1 = sp.expand((even + int(u0)*odd).subs(X, X**2))   # f¹(X²)
lhs, rhs = f1, sp.expand((even + int(u0)*odd).subs(X, X**2))
assert lhs == rhs
print("Constructed f¹(X²) =", f1)
print("✓ Identity holds\n")

# %% [markdown]
# ---
# ## Section 2  Linear Folding in *evaluation* form
# 
# Halving an evaluation vector uses  
# \[
# a'_i \;=\; (1-u_0)a_{2i} + u_0 a_{2i+1}.
# \]

# %%
print("── Section 2 demo ──")
a_eval = [GF(random.randrange(1, 100)) for _ in range(N)]
print("initial eval-vector  a =", a_eval)
a_next = [(1-u0)*a_eval[2*i] + u0*a_eval[2*i+1] for i in range(N//2)]
print("after fixing X₀ =", u0, "→ a' =", a_next, "\n")

# %% [markdown]
# ---
# ## Section 3  Protocol skeleton (commit / open)
# 
# We fake commitments via a SHA-256 digest so we can *see* the values.

# %%
import hashlib, json
def commit(poly):
    blob = json.dumps(str(poly)).encode()
    return hashlib.sha256(blob).hexdigest()[:16]

print("── Section 3  commitment demo ──")
C_f0 = commit(f0)
print("commitment C_f =", C_f0)

beta = GF(random.randrange(1, 100))
h0_beta  = eval_poly(f0, beta)
h0_nbeta = eval_poly(f0, -beta)
h0_b2    = eval_poly(f0, beta**2)
print("\nVerifier challenge β =", beta)
print("h⁰(β)  =", h0_beta)
print("h⁰(-β) =", h0_nbeta)
print("h⁰(β²) =", h0_b2)

h1_b2 = (1-u0)*(h0_beta+h0_nbeta)/2 + u0*(h0_beta-h0_nbeta)/(2*beta)
print("derived h¹(β²) via folding =", h1_b2, "\n")

# %% [markdown]
# ---
# ## Section 4  Optimisations: γ-batching, quotient, linearisation
# 
# 1. Aggregate all $h^{(i)}$ into one poly with a random γ.  
# 2. Interpolate $h^*$ on \{$β, -β, β²$\}.  
# 3. Show $h-h^*=q·Z_D$ divides exactly.  
# 4. Linearise at a fresh ζ.

# %%
print("── Section 4  γ-aggregation ──")
gamma = GF(random.randrange(1, 100))
print("γ =", gamma)

# build symbolic h¹, h² (reuse evaluation-vectors above for simplicity)
h1_sym = f1.subs(X, sp.symbols('X'))
u1 = GF(1)
a_next2 = [(1-u1)*a_next[2*i] + u1*a_next[2*i+1] for i in range(len(a_next)//2)]
h2_sym = sum(int(a_next2[i])*X**i for i in range(len(a_next2)))
h_agg  = sp.expand(f0 + int(gamma)*h1_sym + int(gamma**2)*h2_sym)
print("aggregated h(X) =", h_agg, "\n")

# Lagrange basis on D = {β, -β, β²}
def L(root, roots):
    num = sp.prod([X - r for r in roots if r != root])
    den = sp.prod([root - r for r in roots if r != root])
    return sp.expand(num / den)

roots = [int(beta), int(-beta), int(beta**2)]
L_beta, L_nbeta, L_b2 = [L(r, roots) for r in roots]
print("Lβ  (X) =", sp.expand(L_beta))
print("L-β (X) =", sp.expand(L_nbeta))
print("Lβ²(X) =", sp.expand(L_b2), "\n")

# interpolant h*
v_beta  = sp.poly(h_agg).eval(roots[0])
v_nbeta = sp.poly(h_agg).eval(roots[1])
v_b2    = sp.poly(h_agg).eval(roots[2])
h_star  = sp.expand(v_beta*L_beta + v_nbeta*L_nbeta + v_b2*L_b2)
print("h*(X) =", h_star, "\n")

# quotient q(X)
Z_D = (X-int(beta))*(X+int(beta))*(X-int(beta**2))
q, rem = sp.div(sp.expand(h_agg - h_star), Z_D, domain=sp.QQ)
print("q(X) =", q)
print("remainder after division =", rem)
assert rem == 0
print("✓ h - h* divisible by Z_D\n")

# linearisation at ζ
zeta = GF(12)
Z_D_zeta = (zeta-beta)*(zeta+beta)*(zeta-beta**2)
r_z = sp.expand(h_agg - h_star.subs(X, int(zeta)) - int(Z_D_zeta)*q)
print("ζ =", zeta, "→ Z_D(ζ) =", Z_D_zeta)
print("r_ζ(X) =", r_z)
assert eval_poly(r_z, zeta) == 0
print("✓ r_ζ(ζ) = 0 (vanishes) – ready for witness quotient\n")


── Section 1 demo ──
f⁰(X) = 52*X**7 + 63*X**6 + 66*X**5 + 34*X**4 + 6*X**3 + 54*X**2 + 98*X + 50
even part = 63*X**6 + 34*X**4 + 54*X**2 + 50
odd  part = 52*X**6 + 66*X**4 + 6*X**2 + 98
u₀ = 1
Constructed f¹(X²) = 115*X**12 + 100*X**8 + 60*X**4 + 148
✓ Identity holds

── Section 2 demo ──
initial eval-vector  a = [39, 62, 46, 75, 28, 65, 18, 37]
after fixing X₀ = 1 → a' = [62, 75, 65, 37] 

── Section 3  commitment demo ──
commitment C_f = 086c66cd15162664

Verifier challenge β = 18
h⁰(β)  = 69
h⁰(-β) = 0
h⁰(β²) = 80
derived h¹(β²) via folding = 44 

── Section 4  γ-aggregation ──
γ = 97
aggregated h(X) = 11155*X**12 + 9700*X**8 + 52*X**7 + 63*X**6 + 66*X**5 + 5854*X**4 + 6*X**3 + 54*X**2 + 690*X + 15606 

Lβ  (X) = -X**2/108 + X/36 + 7/2
L-β (X) = X**2/1404 - X/36 + 7/26
Lβ²(X) = X**2/117 - 36/13 

h*(X) = 591026169899581110*X**2 + 1775566698*X - 178587918091280366442 

q(X) = 11155*X**9 + 234255*X**8 + 8533575*X**7 + 179205075*X**6 + 4934323555*X**5 + 103620794707*X**4 + 25554461904